# ALeRCE watchlists, list of coordinates

```Author: Alejandra Muñoz Arancibia```

ALeRCE notebook

Read a list of RA Dec coordinates and search radii, connect to the database and check if there are objects matching a cone search. Check which of the objects have recent detections (date criterion can be changed). Save the lists of objects that match the cone search and meet the date criterion to csv files, including their information from the ``objects`` table

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
from astropy.time import Time
import psycopg2
from datetime import date, datetime, timedelta
import json

Define directory and input file names

In [3]:
dir_watchlists = 'example_tables/' # can be changed to preferred directory
namefil = dir_watchlists+'coords_example.csv' # can be changed to preferred file

Input csv file structure: 1 line header, including name, RA (in deg), Dec (in deg) and search radius (in arcsec)

In [4]:
coords = pd.read_csv(namefil, sep=',', comment='#', skip_blank_lines=True)
coords['r_deg'] = coords['r_arcsec']/3600.
coords

,name,ra,dec,r_arcsec,r_deg
0,WISEAJ175530.00+301326.0,268.87508,30.22383,50.0,0.013889
1,KUG0303-003,46.45086,-0.17677,50.0,0.013889
2,IC2974,178.45304,-5.16783,50.0,0.013889
3,WISEAJ034717.72-082702.9,56.82375,-8.45089,50.0,0.013889
4,CGCG064-083,152.50105,11.91656,50.0,0.013889
5,MCG-01-30-038,177.65333,-7.60794,50.0,0.013889


Connect to the DB

In [5]:
dir_dbfile = '../usecases/' # can be changed to preferred directory
credentials_file = dir_dbfile+'alercereaduser_v2.json' # online data
#credentials_file = dir_dbfile+'alercereaduser_v3.json' # new db

with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'],
                        password=params['password'])

Perform cone search

In [6]:
objs_tot = pd.DataFrame(columns=['name', 'ra', 'dec', 'r_deg', 'oid', 'nobs', 'meanra',
                                 'meandec', 'deltajd', 'lastmjd', 'firstmjd'])

for (name, ra, dec, r_deg) in zip(coords.name, coords.ra, coords.dec, coords.r_deg):
    query='''
    SELECT
        oid, nobs, meanra, meandec, deltajd, lastmjd, firstmjd
    FROM
        objects
    WHERE
        q3c_radial_query(meanra, meandec, %g, %g, %g)
    ''' % (ra, dec, r_deg)
    
    objs_indb = pd.read_sql_query(query, conn)
    objs_indb.set_index('oid', inplace=True)
    nobjs_indb = len(objs_indb)
    print(name, 'Number of oids found in db:', nobjs_indb)
    
    if nobjs_indb>0:
        objs_aux = objs_indb.copy()
        objs_aux = objs_aux.reset_index(level=0)
        objs_aux['name'] = name
        objs_aux['ra'] = ra
        objs_aux['dec'] = dec
        objs_aux['r_deg'] = r_deg
        objs_tot = pd.concat([objs_tot, objs_aux], ignore_index=True)

WISEAJ175530.00+301326.0 Number of oids found in db: 1
KUG0303-003 Number of oids found in db: 2
IC2974 Number of oids found in db: 1
WISEAJ034717.72-082702.9 Number of oids found in db: 3
CGCG064-083 Number of oids found in db: 2
MCG-01-30-038 Number of oids found in db: 4


Display output dataframe

In [7]:
objs_tot
#objs_tot.head()

,name,ra,dec,r_deg,oid,nobs,meanra,meandec,deltajd,lastmjd,firstmjd
0,WISEAJ175530.00+301326.0,268.87508,30.22383,0.013889,ZTF20abxgxjd,42,268.876317,30.222243,46.927280,59137.112199,59090.184919
1,KUG0303-003,46.45086,-0.17677,0.013889,ZTF19aagynyt,1,46.445394,-0.188618,0.000000,58514.147465,58514.147465
2,KUG0303-003,46.45086,-0.17677,0.013889,ZTF18acfwtjw,1,46.447054,-0.189474,0.000000,58431.237940,58431.237940
3,IC2974,178.45304,-5.16783,0.013889,ZTF20acounxo,5,178.461030,-5.171459,7.078600,59180.562280,59173.483681
4,WISEAJ034717.72-082702.9,56.82375,-8.45089,0.013889,ZTF18acckgdl,1,56.810991,-8.448439,0.000000,58425.332813,58425.332813
5,WISEAJ034717.72-082702.9,56.82375,-8.45089,0.013889,ZTF19acchjhf,5,56.823804,-8.450863,409.954630,59169.330903,58759.376273
6,WISEAJ034717.72-082702.9,56.82375,-8.45089,0.013889,ZTF20actekrw,3,56.825589,-8.451758,2.020914,59178.318218,59176.297303
7,CGCG064-083,152.50105,11.91656,0.013889,ZTF19acxpwlq,1,152.493352,11.918123,0.000000,58827.545972,58827.545972
8,CGCG064-083,152.50105,11.91656,0.013889,ZTF20actkmks,3,152.504300,11.916155,2.972083,59179.463588,59176.491505
9,MCG-01-30-038,177.65333,-7.60794,0.013889,ZTF19aanwkpn,1,177.646822,-7.600501,0.000000,58573.255058,58573.255058


Check if oids are unique (without altering the original dataframe)

In [8]:
objs_tot2 = objs_tot[['oid']]

print('Number of oids:', len(objs_tot2))

objs_uniq = pd.unique(objs_tot2['oid'])
objs_tot2 = pd.DataFrame(objs_uniq, columns=['oid'])
nobjs = len(objs_tot2)

display(objs_tot2.head())
print('Number of unique oids:', nobjs)

Number of oids: 13


,oid
0,ZTF20abxgxjd
1,ZTF19aagynyt
2,ZTF18acfwtjw
3,ZTF20acounxo
4,ZTF18acckgdl


Number of unique oids: 13


Find current datetime in local and UTC formats

In [9]:
date_now_local = datetime.now()
offset_utc = 3. # use 3 (4) for Chile Summer (Winter) time, change for your time zone
date_now_utc = date_now_local+timedelta(hours=offset_utc)

date_now_local = date_now_local.strftime('%Y-%m-%dT%H:%M:%S')
date_now_utc = date_now_utc.strftime('%Y-%m-%dT%H:%M:%S')

date_now_local, date_now_utc

('2020-11-27T12:54:30', '2020-11-27T15:54:30')

Save dataframe to csv file, file name will include the current datetime (defined above) in UTC format

In [10]:
dir_out = 'out/' # can be changed to preferred directory

date_now_utc = Time(date_now_utc, format='isot', scale='utc')

namein = namefil.replace(dir_watchlists, '')
namein = namein.replace('.csv', '_out_')

namefil2 = dir_out+namein+str(date_now_utc).replace(':', '_')+'.csv' # can be changed to preferred file name
objs_tot.to_csv(namefil2, index=False)

print('Saved to file:', namefil2)

Saved to file: out/coords_example_out_2020-11-27T15_54_30.000.csv


Show Explorer link for each object. Allows to see the light curve, stamps, Aladin images, etc

In [11]:
for oid in objs_tot['oid']:
    display(HTML("<a href='http://alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid))) # online data
    #display(HTML("<a href='http://dev.alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid))) # new Explorer

Keep a copy of the previous dataframe

In [12]:
objs_tot_1 = objs_tot.copy()

Define date criterion. Example: last 10 days (in MJD)

In [13]:
ndays_lim = 10 # can be changed to preferred date criterion

date_lim = Time(date_now_utc, format="isot", scale="utc").mjd
date_lim = date_lim-ndays_lim

date_lim

59170.66284722222

Apply date criterion

In [14]:
objs_tot = objs_tot_1[objs_tot_1['lastmjd']>=date_lim]
print('Number of oids:', len(objs_tot))
objs_tot.head()

Number of oids: 4


,name,ra,dec,r_deg,oid,nobs,meanra,meandec,deltajd,lastmjd,firstmjd
3,IC2974,178.45304,-5.16783,0.013889,ZTF20acounxo,5,178.461030,-5.171459,7.078600,59180.562280,59173.483681
6,WISEAJ034717.72-082702.9,56.82375,-8.45089,0.013889,ZTF20actekrw,3,56.825589,-8.451758,2.020914,59178.318218,59176.297303
8,CGCG064-083,152.50105,11.91656,0.013889,ZTF20actkmks,3,152.504300,11.916155,2.972083,59179.463588,59176.491505
12,MCG-01-30-038,177.65333,-7.60794,0.013889,ZTF20actqtfk,2,177.651865,-7.606346,1.984190,59178.504352,59176.520162


Sort dataframe by last MJD where there are detections for each object, starting with most recent

In [15]:
col_to_sort = 'lastmjd' # can be changed to preferred column
ascending = False # if True, sorts from smallest to largest - can be changed to preferred sort

objs_tot_sort = objs_tot.sort_values(by=[col_to_sort], ascending=ascending)
objs_tot_sort.head(10)

,name,ra,dec,r_deg,oid,nobs,meanra,meandec,deltajd,lastmjd,firstmjd
3,IC2974,178.45304,-5.16783,0.013889,ZTF20acounxo,5,178.461030,-5.171459,7.078600,59180.562280,59173.483681
8,CGCG064-083,152.50105,11.91656,0.013889,ZTF20actkmks,3,152.504300,11.916155,2.972083,59179.463588,59176.491505
12,MCG-01-30-038,177.65333,-7.60794,0.013889,ZTF20actqtfk,2,177.651865,-7.606346,1.984190,59178.504352,59176.520162
6,WISEAJ034717.72-082702.9,56.82375,-8.45089,0.013889,ZTF20actekrw,3,56.825589,-8.451758,2.020914,59178.318218,59176.297303


Show Explorer link for each object. Allows to see the light curve, stamps, Aladin images, etc (useful for small lists)

In [16]:
for oid in objs_tot_sort['oid']:
    display(HTML("<a href='http://alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid))) # online data
    #display(HTML("<a href='http://dev.alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid))) # new Explorer

Save dataframe to csv file, file name will include the current datetime (defined above) in UTC format and date criterion

In [17]:
namefil3 = dir_out+namein+str(date_now_utc).replace(':', '_')+'_last'+str(int(ndays_lim))+'days.csv' # can be changed to preferred file name
objs_tot_sort.to_csv(namefil3, index=False)

print('Saved to file:', namefil3)

Saved to file: out/coords_example_out_2020-11-27T15_54_30.000_last10days.csv


Close connection to DB

In [18]:
conn.close()